In [1]:
import nibabel as nb
import numpy as np
import subprocess

from dipy.core.gradients import gradient_table
from dipy.data import get_fnames, default_sphere
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti
from pathlib import Path

work_dir = Path('/scratch/jdrussell3/dipy')
orig_dir = work_dir / 'orig'
subjroot = 'sub-011_ses-01'
input_mif = orig_dir / (subjroot + '_ppd.mif')
mask_mif = orig_dir / (subjroot + '_mask_ppd.mif')
input_bvec = str(orig_dir / (subjroot + '_ppd.bvec'))
input_bval = str(orig_dir / (subjroot + '_ppd.bval'))
input_dwi = work_dir / (subjroot + '_ppd.nii')
input_mask = work_dir / (subjroot + '_mask_ppd.nii')

subprocess.run(['mrconvert', input_mif, input_dwi])
subprocess.run(['mrconvert', mask_mif, input_mask])

data, affine, img = load_nifti(input_dwi, return_img=True)
mask_data, affine = load_nifti(input_mask)
bvals, bvecs = read_bvals_bvecs(input_bval, input_bvec)
gtab = gradient_table(bvals, bvecs)
sform = img.get_sform()
qform = img.get_qform()

from dipy.reconst.csdeconv import auto_response
response, ratio = auto_response(gtab, data, roi_radius=10, fa_thr=0.7)
print(response)
print(ratio)

(array([0.00178826, 0.0003949 , 0.0003949 ]), 2060.4368)
0.2208276824486483


In [3]:
from dipy.reconst.csdeconv import ConstrainedSphericalDeconvModel
csd_model = ConstrainedSphericalDeconvModel(gtab, response)
csd_fit = csd_model.fit(data)

KeyboardInterrupt: 